In [1]:
#!/usr/bin/evn python

"""
CMSC733 Spring 2019: Classical and Deep Learning Approaches for
Geometric Computer Vision
Project1: MyAutoPano: Phase 1 Starter Code

Author(s): 
Chahat Deep Singh (chahat@terpmail.umd.edu) 
PhD Student in Computer Science,
University of Maryland, College Park

Nitin J. Sanket (nitinsan@terpmail.umd.edu)
PhD Candidate in Computer Science,
University of Maryland, College Park
"""

#Code starts here:
import numpy as np
import cv2
from skimage.feature import peak_local_max

In [2]:
def readImageSet(folder_name):
    print("Reading images from ", folder_name)
    images = []
    files = os.listdir(folder_name)	
    print("Found ", files)
    for file in files:
	    image_path = folder_name + "/" + file
	    image = cv2.imread(image_path)
	    if image is not None:
		    images.append(image)			
	    else:
		    print("Error in loading image ", image)

    return images


In [3]:
def displayImages(image_array, name):

    concat = image_array[0]
    for l in range(1,len(image_array)):
        concat = np.concatenate((concat, image_array[l]), axis = 1)
    cv2.imshow(name, concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [4]:
def detectCorners(images, choice):
    print("detecting corners ...")
    detected_corners = []
    cmaps = []
    corner_images = []
    for image in images:
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        gray_image = np.float32(gray_image)


        if(choice == 1):
            print("using Harris corner detection method.")
            corner_strength = cv2.cornerHarris(gray_image,2,3,0.04)
            corner_strength[corner_strength<0.01*corner_strength.max()] = 0
            detected_corner = np.where(corner_strength>0.001*corner_strength.max())
            detected_corners.append(detected_corner)
            cmaps.append(corner_strength)
            image[corner_strength > 0.001*corner_strength.max()]=[0,0,255]
            corner_images.append(image)
        else:
            print("using Shi-Tomashi corner detection method.")
            dst = cv2.goodFeaturesToTrack(gray_image,250 ,0.01, 10)
            dst = np.int0(dst)
            detected_corners.append(dst)
            for c in dst:
                x,y = c.ravel()
                cv2.circle(image,(x,y),3,(0, 0, 255),-1) 
                          
            corner_images.append(image)
            cmap = np.zeros(gray_image.shape) #not sure what to do
            cmaps.append(cmap)

    return detected_corners, cmaps, corner_images

In [101]:
def AdaptiveNonMaximalSuppression(images, C_maps, N_best):

    anms_img = []
    anms_corners = []
    for i,image in enumerate(images):

        cmap = C_maps[i]
        locmax = peak_local_max(cmap, min_distance=10)
        n_strong = locmax.shape[0]
        
        r = [np.Infinity for i in range(n_strong)]
        x=np.zeros((n_strong,1))
        y=np.zeros((n_strong,1))
        eu_dist = 0

        for i in range(n_strong):
            for j in range(n_strong):
                x_j = locmax[j][0]
                y_j = locmax[j][1]

                x_i = locmax[i][0]
                y_i = locmax[i][1]

                if(cmap[x_j, y_j] > cmap[x_i, y_i]):
                    eu_dist = np.square(x_j - x_i) + np.square(y_j - y_i)
                if r[i] > eu_dist:
                    r[i] = eu_dist
                    x[i] = x_j
                    y[i] = y_j

        index = np.argsort(r)
        index = np.flip(index)
        index = index[0:N_best]
        x_best=np.zeros((N_best,1))
        y_best=np.zeros((N_best,1))

        for i in range(N_best):
            x_best[i] = np.int0(x[index[i]])
            y_best[i] = np.int0(y[index[i]]) 
            cv2.circle(image, (y_best[i], x_best[i]), 3, (0, 255, 0), -1)

        anms_corner = [x_best, y_best]
        anms_corners.append(anms_corner)
        anms_img.append(image)
    return anms_corners, anms_img


In [102]:
def main():
    # Add any Command Line arguments here
    # Parser = argparse.ArgumentParser()
    # Parser.add_argument('--NumFeatures', default=100, help='Number of best features to extract from each image, Default:100')
    
    # Args = Parser.parse_args()
    # NumFeatures = Args.NumFeatures

    """
    Read a set of images for Panorama stitching
    """
    print("main")
    folder_name = "/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1"
    use_harris = True
    images = readImageSet(folder_name)
    displayImages(images, "image_set")

    """
    Corner Detection
    Save Corner detection output as corners.png
    1 = harris, 2 = shi-tomashi
    """
    if use_harris:
        choice = 1
    else:
        choice = 2

    detected_corners, cmaps, corner_images = detectCorners(images, choice)
    displayImages(corner_images, "corners")
    anms_corners, anms_image = AdaptiveNonMaximalSuppression(images, cmaps, 100)
    displayImages(anms_image, "anms_output")

    """
    Perform ANMS: Adaptive Non-Maximal Suppression
    Save ANMS output as anms.png
    """




In [103]:
if __name__ == '__main__':
    main()


main
Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1
Found  ['3.jpg', '1.jpg', '2.jpg']
detecting corners ...
using Harris corner detection method.
using Harris corner detection method.
using Harris corner detection method.
